In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
with open('moliere.txt') as f:
    mylist = f.read().splitlines()
print(mylist[:10])
print(len(mylist))

['Veux-tu que je te dise ? une atteinte secrète', 'Ne laisse point mon âme en une bonne assiette :', 'Oui, quoi qu’à mon amour tu puisses repartir,', 'Il craint d’être la dupe, à ne te point mentir :', 'Qu’en faveur d’un rival ta foi ne se corrompe,', 'Ou du moins, qu’avec moi, toi-même on ne te trompe.', 'Pour moi, me soupçonner de quelque mauvais tour,', 'Je dirai, n’en déplaise à monsieur votre amour,', 'Que c’est injustement blesser ma prud’homie', 'Et se connaître mal en physionomie.']
15462


In [ ]:
tokenizer = Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(mylist)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
for line in mylist:
  token_list = tokenizer.texts_to_sequences([line])[0]
  input_sequences.append(token_list)
print(input_sequences[0])
print(tokenizer.sequences_to_texts([input_sequences[0]]))

18069
[1166, 4, 6, 101, 2759, 10, 44, 1167, 1286]
['Veux-tu que je te dise ? une atteinte secrète']


In [ ]:
input_sequences = np.asarray(input_sequences) 
quatrains = []
cur_quat = []
incr = 0
for line in input_sequences:
  cur_quat.append(line)
  if (incr > 0) & (incr%2 ==1):
    quatrains.append(cur_quat)
    cur_quat = [] 
  incr +=1
print(quatrains[:4])
X_train = quatrains[:][0]
Y_train = quatrains[:][1]
X_train, Y_train = np.array(X_train), np.array(Y_train)
maxlength_x = 0
for x in X_train:
  if len(x) > maxlength_x:
    maxlength_x = len(x)
print(maxlength_x)
maxlength_y = 0
for x in Y_train:
  if len(x) > maxlength_y:
    maxlength_y = len(x)
print(maxlength_y)
'''sequence_length = 20
predictors, label = [], []
for t in range(len(input_sequences1D) - sequence_length):
  predictors.append(input_sequences1D[t: t+sequence_length])
  label.append(input_sequences1D[t+sequence_length])
label = ku.to_categorical(label, num_classes=total_words)
predictors, label = np.asarray(predictors), np.asarray(label)
print(predictors.shape)
print(label.shape)'''

[[[1166, 4, 6, 101, 2759, 10, 44, 1167, 1286], [104, 473, 36, 21, 115, 15, 44, 251, 7440, 24]], [[87, 109, 128, 21, 158, 108, 7441, 4761], [37, 1982, 221, 11, 2760, 7, 12, 101, 36, 2761, 24]], [[816, 586, 56, 587, 344, 294, 12, 47, 7442], [398, 43, 345, 387, 96, 3480, 62, 12, 101, 4762]], [[67, 96, 17, 2311, 1, 81, 444, 1983], [18, 2312, 148, 2762, 7, 2313, 26, 525]]]
10
11


'sequence_length = 20\npredictors, label = [], []\nfor t in range(len(input_sequences1D) - sequence_length):\n  predictors.append(input_sequences1D[t: t+sequence_length])\n  label.append(input_sequences1D[t+sequence_length])\nlabel = ku.to_categorical(label, num_classes=total_words)\npredictors, label = np.asarray(predictors), np.asarray(label)\nprint(predictors.shape)\nprint(label.shape)'

In [ ]:
from tensorflow.keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen = maxlength_x)
Y_train = sequence.pad_sequences(Y_train, maxlen = maxlength_y)

In [ ]:
model = Sequential(
        [
         Embedding(total_words, 64, input_length=maxlength_x),
         LSTM(256, return_sequences=True), Dropout(0.1),
         LSTM(256),
         Dense(total_words, activation='softmax')
        ]
    )
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 64)            1156416   
_________________________________________________________________
lstm (LSTM)                  (None, 20, 256)           328704    
_________________________________________________________________
dropout (Dropout)            (None, 20, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 18069)             4643733   
Total params: 6,654,165
Trainable params: 6,654,165
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(predictors, label, epochs=40, batch_size=64)

Epoch 1/40
2090/2090 [==============================] - 48s 23ms/step - loss: 7.6710 - accuracy: 0.0289
Epoch 2/40
2090/2090 [==============================] - 48s 23ms/step - loss: 6.9945 - accuracy: 0.0446
Epoch 3/40
2090/2090 [==============================] - 48s 23ms/step - loss: 6.5531 - accuracy: 0.0635
Epoch 4/40
2090/2090 [==============================] - 48s 23ms/step - loss: 6.1675 - accuracy: 0.0809
Epoch 5/40
2090/2090 [==============================] - 48s 23ms/step - loss: 5.8097 - accuracy: 0.0953
Epoch 6/40
2090/2090 [==============================] - 48s 23ms/step - loss: 5.4658 - accuracy: 0.1059
Epoch 7/40
2090/2090 [==============================] - 48s 23ms/step - loss: 5.1542 - accuracy: 0.1138
Epoch 8/40
2090/2090 [==============================] - 48s 23ms/step - loss: 4.8568 - accuracy: 0.1223
Epoch 9/40
2090/2090 [==============================] - 49s 23ms/step - loss: 4.5746 - accuracy: 0.1348
Epoch 10/40
2090/2090 [==============================] - 49s 23m

In [ ]:
pattern = predictors[0]
print(pattern)
print(pattern.shape)
print(np.expand_dims(pattern, 0).shape)

[1166    4    6  101 2759   10   44 1167 1286  104  473   36   21  115
   15   44  251 7440   24   87]
(20,)
(1, 20)


In [ ]:
# --- note_l: is the list of notes (integer number) over time
note_l = [k for k in pattern]
# --- prediction_l: is the list of output-vectors (float numbers \in [0,1]) of the network over time
prediction_l = []
# --- Generate T_y_generated notes
for note_index in range(62):
        # --- START CODE HERE (03)
        prediction = model.predict(np.expand_dims(pattern, 0))
        prediction_l.append(prediction)
        index = np.argmax(prediction[0])
        print(index)
        note_l.append(index)
        pattern = np.concatenate((pattern[1:],[index]), axis=0)
        # --- END CODE HERE

109
128
35
17881
17882
53
17883
85
700
22
922
21
1218
1
8290
5137
170
33
77
36
1
257
7
205
113
2097
2752
98
13
640
1248
10
875
20
1
174
406
6
72
743
1
2
10
18
12
220
25
712
4517
2533
62
2
410
29
26
115
15
158
17
466
896
5


In [ ]:
print(tokenizer.sequences_to_texts([note_l]))

['Veux-tu que je te dise ? une atteinte secrète Ne laisse point mon âme en une bonne assiette : Oui, quoi qu’à son Crédit nuise cette Aventure, Ce chagrin est céans mon frère de sable… Quand… quand il n’est point de part à toutes deux L’autre Nymphes, Monsieur, un peu, laissons ? Las ! de leurs vœux, je suis sûr de vous ? Je ne dis pas cela. Nul obstacle, on vous plaît, Que votre âme en amour me pourra apprendre ;']


# Ponctuation et casse : entrainement par séquences de 20 tokens

**2 couches LSTM, 20 epochs**

Veux-tu que je te dise ? une atteinte secrète 

Ne laisse point mon âme en une bonne assiette : 

Oui, Madame, cessez de la sorte expliqués… 

Je ne sais pas, Monsieur, de vous le figurer ; 

Et que de la négliger, il faut bien vécu, 

Il est vrai que je vous ai rencontré la tête. 

Oui, oui, je vous commande avec beaucoup de colère. 

Oh… Non, je ne puis point séduire ? 

Je ne sais pas cela que je vous ai portée.

**35 epochs**

Oui, quoi qu’à mon cœur devrait rompre, de la répandre, 

Et goûtât-on cela, s’il vous plaît, que je meure. 

Comment ? Vous moquez-vous ? Monsieur, perdez cette matière, 

Je reviens de vous payer, Témoins de la fenêtre : 

Madame ; je suis toute ébaubie, et je vois… Franc abus. 

Pour le Siècle passé dans leurs productions, 

Vous merveilleux des plaisirs de la nature

**40 epochs**

Oui, quoi qu’à son Crédit nuise cette Aventure, 

Ce chagrin est céans mon frère de sable… 

Quand… quand il n’est point de part à toutes deux 

L’autre Nymphes, Monsieur, un peu, laissons ? 

Las ! de leurs vœux, je suis sûr de vous ? 

Je ne dis pas cela. Nul obstacle, on vous plaît, 

Que votre âme en amour me pourra apprendre ;

# Many-to-One

**2 couches LSTM, 100 epochs**

A peine les ont de leurs baisers de ces coeurs, 

De la seine et du plus m a mal affreux le bruit ;

à chaque éternité le ciel le la ermite

**2 couches LSTM, 200 epochs**

Le navire glissant sur son coeur qui ne me

Dit jamais bien,  beaux vie et les seins nous de

Reine cette coulisse, où mon esprit n est un

Cher métier comme une fleur, un pauvre qui est-il ?

**3 couches LSTM, 100 epochs**

Dans la maison et les soleils funèbres pour

pleins en amante dans le coeur sans nuages et

non où jette en volant ton cou s verse d belle je

comme qu elle remue avec la main et regard le

**3 couches LSTM, 200 epochs**

le navire glissant sur les gouffres amers 

a peine les ont ils déposés sur les planches 

que ces rois de l azur maladroits et honteux 

laissent piteusement leurs grandes ailes blanches

**Plus de dimensions d'embedding :**

Belle comme des chairs de fleurs qui de l enfer

Je suis dis dans des chansons les coeurs et des yeux

Et ô ma chère sans doute et troubler en ton coeur

La retrouve des ennuis l erèbe les eût pris

